In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.model_selection import train_test_split

In [2]:
X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR = r"/content/drive/MyDrive/flowers/daisy"
FLOWER_SUNFLOWER_DIR=r"/content/drive/MyDrive/flowers/sunflower"
FLOWER_TULIP_DIR=r"/content/drive/MyDrive/flowers/tulip"
FLOWER_DANDI_DIR=r"/content/drive/MyDrive/flowers/dandelion"
FLOWER_ROSE_DIR=r"/content/drive/MyDrive/flowers/rose"

In [3]:
def assign_label(img, flower_type):
    if flower_type == 'daisy':
        return 0
    elif flower_type == 'sunflower':
        return 1
    elif flower_type == 'tulip':
        return 2
    elif flower_type == 'dandelion':
        return 3
    elif flower_type == 'rose':
        return 4
    else:
        return -1  # Unknown flower type


In [4]:
def make_train_data(flower_type, DIR):
    for img in tqdm(os.listdir(DIR)):
        label = assign_label(img, flower_type)
        if label != -1:
            path = os.path.join(DIR, img)
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize images to 150x150 pixels
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert images to RGB format
            img = img.astype(np.float32) / 255.0  # Normalize pixel values to range [0, 1]
            X.append(np.array(img))
            Z.append(label)

In [5]:
# Call make_train_data function for each flower type
make_train_data('daisy', FLOWER_DAISY_DIR)
make_train_data('sunflower', FLOWER_SUNFLOWER_DIR)
make_train_data('tulip', FLOWER_TULIP_DIR)
make_train_data('dandelion', FLOWER_DANDI_DIR)
make_train_data('rose', FLOWER_ROSE_DIR)

# Total number of records
total_records = len(X)

100%|██████████| 784/784 [00:17<00:00, 44.12it/s] 


In [6]:
le = LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)

In [7]:
# Split the data into training and test sets (75/25 ratio)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Print the shapes of the training and test sets
print("Training set:")
print("X_train shape:", np.array(X_train).shape)
print("Y_train shape:", np.array(Y_train).shape)
print("\nTest set:")
print("X_test shape:", np.array(X_test).shape)
print("Y_test shape:", np.array(Y_test).shape)


Training set:
X_train shape: (3245, 150, 150, 3)
Y_train shape: (3245, 5)

Test set:
X_test shape: (1082, 150, 150, 3)
Y_test shape: (1082, 5)


In [8]:
# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

<ipython-input-8-3c35342518d2>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  X_train = torch.tensor(X_train, dtype=torch.float32)


In [9]:
# Reshape input tensors to (batch_size, 3, 150, 150)
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [10]:
# Load pre-trained ResNet model
pretrained_resnet = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


In [11]:
# Modify the fully connected layer for flower classification
num_ftrs = pretrained_resnet.fc.in_features
pretrained_resnet.fc = nn.Linear(num_ftrs, 5)  # Assuming 5 flower classes

In [12]:
# Freeze convolutional layers
for param in pretrained_resnet.parameters():
    param.requires_grad = False

In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_resnet.fc.parameters(), lr=0.001)

In [14]:
# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_resnet.to(device)
X_train, X_test, Y_train, Y_test = X_train.to(device), X_test.to(device), Y_train.to(device), Y_test.to(device)

In [15]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    pretrained_resnet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in zip(X_train, Y_train):
        optimizer.zero_grad()

        # Add batch dimension to input tensor
        inputs = inputs.unsqueeze(0)

        # Convert input tensor to floating point type and ensure it requires gradients
        inputs = inputs.float()
        inputs.requires_grad = True

        outputs = pretrained_resnet(inputs)
        _, predicted = torch.max(outputs, 1)  # Get predicted class indices

        # Add batch dimension to labels tensor and convert to LongTensor
        labels = labels.unsqueeze(0).float()

        # Compute loss directly using class indices (no need to convert labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Compare with original labels
        running_loss += loss.item()

    train_loss = running_loss / total
    train_accuracy = correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")


Epoch [1/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [2/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [3/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [4/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [5/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [6/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [7/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [8/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [9/10], Train Loss: 1.6098, Train Accuracy: 3.9914
Epoch [10/10], Train Loss: 1.6098, Train Accuracy: 3.9914


In [16]:
# Evaluate the model
pretrained_resnet.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in zip(X_test, Y_test):
        inputs, labels = inputs.unsqueeze(0), labels.unsqueeze(0)
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = pretrained_resnet(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(torch.argmax(labels)).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.1645


FOR TENSORFLOW

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, Model

In [18]:
# Load the pre-trained ResNet50 model excluding the top layers
base_model = tf.keras.applications.ResNet50(input_shape=(150, 150, 3),
                                            include_top=False,
                                            weights='imagenet')

94765736/94765736 [==============================] - 0s 0us/step


In [19]:
# Freeze the pre-trained layers
base_model.trainable = False

In [20]:
# Add a new fully connected layer for flower classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(5, activation='softmax')(x)  # Assuming 5 flower classes

In [21]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [25]:
# Convert data to NumPy arrays
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)
Y_train_np = np.array(Y_train)
Y_test_np = np.array(Y_test)

In [26]:
# Ensure the data sizes match
assert len(X_train_np) == len(Y_train_np), "Training data and labels have different sizes"
assert len(X_test_np) == len(Y_test_np), "Test data and labels have different sizes"

In [27]:
# Train the model
model.fit(X_train_np, Y_train_np, epochs=20, batch_size=32, validation_data=(X_test_np, Y_test_np))

Epoch 1/20
102/102 [==============================] - 19s 112ms/step - loss: 1.5406 - accuracy: 0.3174 - val_loss: 1.4814 - val_accuracy: 0.3577
Epoch 2/20
102/102 [==============================] - 7s 69ms/step - loss: 1.4831 - accuracy: 0.3683 - val_loss: 1.4391 - val_accuracy: 0.4067
Epoch 3/20
102/102 [==============================] - 8s 81ms/step - loss: 1.4431 - accuracy: 0.3840 - val_loss: 1.4151 - val_accuracy: 0.4316
Epoch 4/20
102/102 [==============================] - 8s 79ms/step - loss: 1.4139 - accuracy: 0.4086 - val_loss: 1.3678 - val_accuracy: 0.4418
Epoch 5/20
102/102 [==============================] - 7s 70ms/step - loss: 1.3962 - accuracy: 0.4194 - val_loss: 1.3914 - val_accuracy: 0.4603
Epoch 6/20
102/102 [==============================] - 7s 71ms/step - loss: 1.3620 - accuracy: 0.4579 - val_loss: 1.3359 - val_accuracy: 0.4501
Epoch 7/20
102/102 [==============================] - 8s 78ms/step - loss: 1.3530 - accuracy: 0.4330 - val_loss: 1.3515 - val_accuracy: 0.42

In [29]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_np, Y_test_np)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

34/34 [==============================] - 2s 52ms/step - loss: 1.2383 - accuracy: 0.5055
Test Loss: 1.2382563352584839, Test Accuracy: 0.5055452585220337
